In [2]:
import os
import sqlite3
import pandas as pd
import logging

# Configure logging
LOGS_PATH = "logs/"
os.makedirs(LOGS_PATH, exist_ok=True)
logging.basicConfig(filename=os.path.join(LOGS_PATH, 'feature_store.log'), level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define database path
DB_PATH = "database/feature_store.db"
os.makedirs("database", exist_ok=True)

# Connect to SQLite database
def get_db_connection():
    return sqlite3.connect(DB_PATH)

# Initialize feature store
def initialize_feature_store():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Create feature metadata table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS feature_metadata (
            feature_name TEXT PRIMARY KEY,
            description TEXT,
            data_type TEXT,
            source TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    conn.close()
    logging.info("Feature store initialized.")

# Store features in the feature store
def store_features(df, table_name):
    conn = get_db_connection()
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    logging.info(f"Features stored in {table_name} table.")

# Retrieve features for model training
def get_features(table_name):
    conn = get_db_connection()
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

if __name__ == "__main__":
    initialize_feature_store()
    
    # Example usage: Storing and retrieving features
    sample_data = {
        'customer_id': [1, 2, 3],
        'total_spend': [500.0, 1200.5, 800.3],
        'tenure_months': [12, 24, 36]
    }
    df = pd.DataFrame(sample_data)
    store_features(df, "customer_features")
    
    retrieved_df = get_features("customer_features")
    print(retrieved_df)

   customer_id  total_spend  tenure_months
0            1        500.0             12
1            2       1200.5             24
2            3        800.3             36
